In [11]:
import sys
sys.path.append('../code')
from resnet import *
from cifar_very_tiny import *
from cifar_dataset import *    
import torch as t 
import numpy as np
import tqdm
import matplotlib.pylab as plt
import matplotlib.cm as cm
import json

%matplotlib inline
plt.rcParams['figure.figsize']=(12,9)
plt.rcParams['font.size']= 20

In [2]:
train_loader, test_loader, train_loader_no_augumentation = cifar10_loader(batch_size=128,  maxsize=10112)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
epoch_num = 50
basic_results = [] # результаты без дистилляции. Каждый элемент списка - accuracy
resnet_results = [] # результаты с дистилляцией ResNet. Каждый элемент списка - кортеж вида (accuracy, beta, temp)
cnn_results = [] # результаты с дистилляцией CNN. Каждый элемент списка - кортеж вида (accuracy, beta, temp)

# версия нужна, чтобы различать старые и новые результаты экспериментов. 
# менять нужно каждый раз, когда есть хотя бы незначительные изменения в эксперименте
experiment_version = '2' 


In [4]:
def accuracy(student):
        total = 0 
        correct = 0
        with t.no_grad():
            for x,y in test_loader:
                x = x.to(device)
                y = y.to(device)
                out = student(x)
                correct += t.eq(t.argmax(out, 1), y).sum()
                total+=len(x)
        return (correct/total).cpu().detach().numpy()

In [5]:
# 5 запусков --- без дистилляции

In [5]:
for _ in range(5):
    student = Cifar_Very_Tiny(10).to(device)
    optim = t.optim.Adam(student.parameters())
    #optim = t.optim.SGD(student.parameters(), lr=0.001)
    crit = nn.CrossEntropyLoss()
    for _ in range(epoch_num):
        tq = tqdm.tqdm(train_loader_no_augumentation)
        losses = []
        for x,y in tq:
            x = x.to(device)
            y = y.to(device)
            student.zero_grad()
            out = student(x)
            loss = crit(student(x), y)
            losses.append(loss.cpu().detach().numpy())
            loss.backward()
            optim.step()
            tq.set_description('current loss:{}'.format(np.mean(losses[-10:])))    
    basic_results.append(accuracy(student))
    # сохраняем наши результаты в формате jsonl (каждая строка --- словарь в формате json)
    # обрати внимание, что флаг открытия файла --- 'a', позволяющий дозаписывать результаты
    with open('../log/basic_results.jsonl', 'a') as out:
        out.write(json.dumps({'accuracy':float(basic_results[-1]), 'version': experiment_version})+'\n')
    print ('accuracy', basic_results[-1])

current loss:2.334538459777832:   3%|▎         | 2/79 [00:00<00:04, 17.44it/s] 

accuracy 0.5185


current loss:2.3215205669403076:   3%|▎         | 2/79 [00:00<00:04, 16.25it/s]

accuracy 0.5022


current loss:2.312763214111328:   3%|▎         | 2/79 [00:00<00:04, 16.87it/s]

accuracy 0.5221


current loss:2.3089959621429443:   3%|▎         | 2/79 [00:00<00:04, 17.63it/s]

accuracy 0.5314


current loss:0.46031156182289124: 100%|██████████| 79/79 [00:04<00:00, 18.14it/s]


accuracy 0.5061


In [6]:
kl = nn.KLDivLoss(reduction='batchmean')
sm = nn.Softmax(dim=1)

def distill(out, batch_logits, temp):
    g = sm(out/temp)
    f = F.log_softmax(batch_logits/temp)    
    return kl(f, g)

In [8]:
# 20 запусков --- с ResNet
logits = np.load('./logits_resnet.npy')
for _ in range(20):
    beta = np.random.uniform()
    temp = 10**(np.random.uniform(low=-1, high=1)) # температура от 0.1 до 10
    print ('hyperparameters', beta, temp)
    student = Cifar_Very_Tiny(10).to(device)
    optim = t.optim.Adam(student.parameters())
    #optim = t.optim.SGD(student.parameters(), lr=0.001)
    crit = nn.CrossEntropyLoss()
    for _ in range(epoch_num):
        tq = tqdm.tqdm(train_loader_no_augumentation)
        losses = []
        for batch_id, (x,y) in enumerate(tq):
            x = x.to(device)
            y = y.to(device)
            batch_logits = t.Tensor(logits[128*batch_id:128*(batch_id+1)]).to(device)            
            student.zero_grad()
            out = student(x)
            student_loss = crit(student(x), y)
            #distillation_loss = 0.0 # здесь твой код!
            distillation_loss = distill(out, batch_logits, temp)
            loss = (1-beta) * student_loss + beta*distillation_loss
            losses.append(loss.cpu().detach().numpy())
            loss.backward()
            optim.step()
            tq.set_description('current loss:{}'.format(np.mean(losses[-10:])))
    resnet_results.append((accuracy(student), beta, temp))
    print ('accuracy', resnet_results[-1][0])
    with open('../log/resnet_results.jsonl', 'a') as out:
        out.write(json.dumps({'accuracy':float(resnet_results[-1][0]), 'beta':beta, 'temp':temp, 'version': experiment_version})+'\n')

  0%|          | 0/79 [00:00<?, ?it/s]<ipython-input-6-cea0592a0b8b>:6: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  f = F.log_softmax(batch_logits/temp)
current loss:2.394636869430542:   1%|▏         | 1/79 [00:00<00:08,  9.11it/s]

hyperparameters 0.05636748720019913 5.590773865421679


current loss:73.59500122070312:   3%|▎         | 2/79 [00:00<00:05, 13.79it/s]

accuracy 0.5184
hyperparameters 0.8288778877779225 0.32178697537831397


current loss:10.489887237548828:   3%|▎         | 2/79 [00:00<00:05, 13.69it/s]

...

accuracy 0.5527
hyperparameters 0.3125452766510102 2.8938644687468535


current loss:3.760899782180786:   3%|▎         | 2/79 [00:00<00:05, 12.87it/s]

accuracy 0.5388
hyperparameters 0.807811667640158 4.70350491682745


current loss:1.1020095348358154: 100%|██████████| 79/79 [00:05<00:00, 14.29it/s]


accuracy 0.5571


In [10]:
# 20 запусков --- с CNN
logits = np.load('./logits_cnn.npy')
for _ in range(20):
    beta = np.random.uniform()
    temp = 10**(np.random.uniform(low=-1, high=1)) # температура от 0.1 до 10 
    print ('hyperparameters', beta, temp)
    student = Cifar_Very_Tiny(10).to(device)
    optim = t.optim.Adam(student.parameters())
    #optim = t.optim.SGD(student.parameters(), lr=0.001)
    crit = nn.CrossEntropyLoss()
    for _ in range(epoch_num):
        tq = tqdm.tqdm(train_loader_no_augumentation)
        losses = []
        for batch_id, (x,y) in enumerate(tq):
            x = x.to(device)
            y = y.to(device)            
            batch_logits = t.Tensor(logits[128*batch_id:128*(batch_id+1)]).to(device)            
            student.zero_grad()
            out = student(x)
            student_loss = crit(student(x), y)
            #distillation_loss = 0.0 # здесь твой код!
            distillation_loss = distill(out, batch_logits, temp)
            loss = (1-beta) * student_loss + beta*distillation_loss
            losses.append(loss.cpu().detach().numpy())
            loss.backward()
            optim.step()
            tq.set_description('current loss:{}'.format(np.mean(losses[-10:])))
    cnn_results.append((accuracy(student), beta, temp))
    with open('../log/cnn_results.jsonl', 'a') as out:
        out.write(json.dumps({'accuracy':float(cnn_results[-1][0]), 'beta':beta, 'temp':temp, 'version': experiment_version})+'\n')
    print ('accuracy', cnn_results[-1][0])

  0%|          | 0/79 [00:00<?, ?it/s]<ipython-input-6-cea0592a0b8b>:6: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  f = F.log_softmax(batch_logits/temp)
current loss:13.106579780578613:   1%|▏         | 1/79 [00:00<00:08,  8.85it/s]

hyperparameters 0.1455241640942313 0.1569715254455658


current loss:2.0333051681518555:   3%|▎         | 2/79 [00:00<00:05, 12.93it/s]

accuracy 0.5619
hyperparameters 0.850134684325327 3.1766704293650356


current loss:2.0417654514312744:   3%|▎         | 2/79 [00:00<00:05, 13.75it/s]

...

accuracy 0.5786
hyperparameters 0.2944576309104301 0.5545485480564484


current loss:6.778729438781738:   3%|▎         | 2/79 [00:00<00:05, 13.53it/s]

accuracy 0.5676
hyperparameters 0.7775814610064766 1.1776857459611674


current loss:0.8692722320556641: 100%|██████████| 79/79 [00:05<00:00, 14.40it/s]


accuracy 0.5648


In [20]:
# загружаем данные и проверяем версию экспериментов

basic_results = [] # результаты без дистилляции. Каждый элемент списка - accuracy
resnet_results = [] # результаты с дистилляцией ResNet. Каждый элемент списка - кортеж вида (accuracy, beta, temp)
cnn_results = [] # результаты с дистилляцией CNN. Каждый элемент списка - кортеж вида (accuracy, beta, temp)

with open('../log/basic_results.jsonl', 'r') as inp:
    for line in inp:
        data = json.loads(line)
        if data['version'] == experiment_version:
            basic_results.append(data['accuracy'])

with open('../log/resnet_results.jsonl', 'r') as inp:
    for line in inp:
        data = json.loads(line)
        if data['version'] == experiment_version:
            resnet_results.append((data['accuracy'], data['beta'], data['temp']) )           
            
with open('../log/cnn_results.jsonl', 'r') as inp:
    for line in inp:
        data = json.loads(line)
        if data['version'] == experiment_version:
            cnn_results.append((data['accuracy'], data['beta'], data['temp']) )                       

FileNotFoundError: [Errno 2] No such file or directory: './log/basic_results.jsonl'

In [ ]:
# график, разрез beta-accuracy
plt.scatter([0]*len(basic_results), basic_results, c='r', label='Без дистилляции')
#plt.scatter([r[1] for r in cnn_results], [r[0] for r in cnn_results], c='r', marker='x', label='Дистилляция CNN')
plt.scatter([r[1] for r in resnet_results], [r[0] for r in resnet_results], c='r', marker='d', label='Дистилляция ResNet')
plt.legend(loc='best')
plt.xlabel('Beta')
plt.ylabel('Accuracy')
plt.savefig('../figs/scatter_beta_acc.png')

In [ ]:
# график, разрез accuracy-Temp
plt.scatter([0.0]*len(basic_results), basic_results, c='r', label='Без дистилляции')
#plt.scatter([np.log10(r[2]) for r in cnn_results], [r[0] for r in cnn_results], c='r', marker='x', label='Дистилляция CNN')
plt.scatter([np.log10(r[2]) for r in resnet_results], [r[0] for r in resnet_results], c='r', marker='d', label='Дистилляция ResNet')
plt.legend(loc='best')
plt.xlabel('log(T)')
plt.ylabel('Accuracy')
plt.savefig('../figs/scatter_temp_acc.png')

In [16]:
from matplotlib import pylab as plt
plt.rcParams['font.family'] = 'DejaVu Serif'
plt.rcParams['lines.linewidth'] = 2
plt.rcParams['lines.markersize'] = 12
plt.rcParams['xtick.labelsize'] = 24
plt.rcParams['ytick.labelsize'] = 24
plt.rcParams['legend.fontsize'] = 24
plt.rcParams['axes.titlesize'] = 36
plt.rcParams['axes.labelsize'] = 24

In [18]:
# график, разрез Temp-beta. Цвет --- точность относительно всех результатов

# для резульатов без дистилляции у нас будут повторяющиеся точки. 
# обычно в таком случае в график вносят небольшой шум, чтобы точки можно было визуально различить
eps = 0.02 

all_results = basic_results+[r[0] for r in cnn_results] + [r[0] for r in resnet_results]
max_ = np.max(all_results)
min_ = np.min(all_results)
colors = [cm.seismic((r-min_)/(max_-min_)) for r in basic_results]
plt.scatter(np.random.randn(len(basic_results))*eps, np.random.randn(len(basic_results))*eps, c=colors, label='Без дистилляции')
colors = [cm.seismic((r[0]-min_)/(max_-min_)) for r in cnn_results]
plt.scatter([np.log10(r[2]) + np.random.randn()*eps for r in cnn_results], [r[1] + np.random.randn()*eps for r in cnn_results], c=colors, marker='x', label='Дистилляция CNN')
colors = [cm.seismic((r[0]-min_)/(max_-min_)) for r in resnet_results]
plt.scatter([np.log10(r[2]) + np.random.randn()*eps for r in resnet_results], [r[1] + np.random.randn()*eps for r in resnet_results], c=colors, marker='d', label='Дистилляция ResNet')
plt.legend(loc='upper left')
plt.xlabel('log(T)')
plt.ylabel('Beta')
plt.savefig('../figs/scatter_temp_beta.pdf')

ValueError: zero-size array to reduction operation maximum which has no identity

In [19]:
basic_results

[]